In [ ]:
%load_ext autoreload
%autoreload 2

import firedrake
import icepack
from icepack.constants import ice_density as rho_ice, water_density as rho_sw
from preproc.flow_models import icepack_model,icepack_solver
from preproc.inverse import reg_smooth,create_fwd_model,create_loss,create_reg
from preproc.utils.proj import trunc,get_flowline,output_regular,project_onto_dir
from preproc.utils.plot import tripcolor,ip_streamplot,plot_mesh,plot_flowlines,plot_all_1d
from omegaconf import OmegaConf
from hydra import compose, initialize
import logging
from pathlib import Path
import numpy as np
from scipy.interpolate import splev, splprep
import rasterio
from icepack.statistics import (
    StatisticsProblem,
    MaximumProbabilityEstimator,
)


In [ ]:
log = logging.getLogger(__name__)


def load_mesh(cfg):
    try:
        mesh = firedrake.Mesh(str(Path(cfg.workspace,"data",cfg.shelf,cfg.mesh.mesh_file).absolute()))
    except FileNotFoundError:
        raise FileNotFoundError(f"Mesh file {cfg.mesh_file} not found in {cfg.workspace}/data/{cfg.shelf}")
    
    mesh_family = cfg.mesh.get("mesh_family","CG")
    mesh_degree = cfg.mesh.get("mesh_degree",2)
    if cfg.mesh.dim == 2:
        Q = firedrake.FunctionSpace(mesh,mesh_family,mesh_degree)
        V = firedrake.VectorFunctionSpace(mesh,mesh_family,mesh_degree)
        return mesh,Q,V
    elif cfg.mesh.dim == 3:
        raise NotImplementedError("3D mesh loading not yet implemented")

def load_flowline(cfg):
    try:
        mesh1d = firedrake.Mesh(str(Path(cfg.workspace,"data",cfg.shelf,cfg.flowline.embedded_mesh.mesh_file).absolute()),dim=2)
    except FileNotFoundError:
        raise FileNotFoundError(f"Mesh file {cfg.flowline.embedded_mesh.mesh_file} not found in {cfg.workspace}/data/{cfg.shelf}")
    embedded_mesh_family = cfg.flowline.embedded_mesh.get("mesh_family","CG")
    embedded_mesh_degree = cfg.flowline.embedded_mesh.get("mesh_degree",1)
    V_funcspace = firedrake.VectorFunctionSpace(mesh1d,family=embedded_mesh_family,degree=embedded_mesh_degree,dim=2)
    Q_funcspace = firedrake.FunctionSpace(mesh1d,family=embedded_mesh_family,degree=embedded_mesh_degree)
    x = mesh1d.coordinates.dat.data_ro[:,0]
    y = mesh1d.coordinates.dat.data_ro[:,1]
    coordinates,dist = get_flowline(x,y,**cfg.flowline.smoothing)
    tck,u = splprep(coordinates.T, u=dist,k=3)
    tangent = np.array(splev(u,tck,der=1)).T
    tangent /= np.linalg.norm(tangent,axis=1)[:,None]
    normal = np.vstack([-tangent[:,1],tangent[:,0]]).T
    normal /= np.linalg.norm(normal,axis=1)[:,None]
    return mesh1d,Q_funcspace,V_funcspace,coordinates,dist,tangent,normal

def load_fields(cfg,Q_c,V):
    try:
        thickness = rasterio.open(f'netcdf:{str(Path(cfg.workspace,"data",cfg.data.thickness.fname).absolute())}:thickness', 'r')
    except FileNotFoundError:
        raise FileNotFoundError(f"Thickness file {cfg.data.thickness.fname} not found in {cfg.workspace}/data/{cfg.shelf}")
    h_obs = icepack.interpolate(thickness,Q_c)
    try:
        fname = str(Path(cfg.workspace,"data",cfg.data.velocity.fname).absolute())
        vx = rasterio.open(f'netcdf:{fname}:VX', 'r')
        vy = rasterio.open(f'netcdf:{fname}:VY', 'r')
        stdx = rasterio.open(f'netcdf:{fname}:ERRX', 'r')
        stdy = rasterio.open(f'netcdf:{fname}:ERRY', 'r')
    except FileNotFoundError:
        raise FileNotFoundError(f"Velocity file {cfg.data.velocity.fname} not found in {cfg.workspace}/data/{cfg.shelf}")
    u_obs = icepack.interpolate((vx,vy),V)
    sigma_x = icepack.interpolate(stdx,Q_c)
    sigma_y = icepack.interpolate(stdy,Q_c)

    if cfg.data.thickness.smooth:
        log.info("Smoothing thickness")
        h = reg_smooth(h_obs,cfg.data.thickness.smoothing_alpha,cfg.data.thickness.smoothing_sd)

    
    return h,u_obs,sigma_x,sigma_y

In [ ]:
initialize(config_path="../configs",job_name="test")
cfg = compose(config_name="ekstrom_final")
print(cfg)
if cfg.mesh.dim == 2:
    mesh,Q_c,V = load_mesh(cfg)
elif cfg.mesh.dim == 3:
    mesh2d,Q_c,mesh,Q,V,W = load_mesh(cfg)
log.info("Loaded mesh")
plot_mesh(mesh)


mesh1d,Q_emb,V_emb,coordinates,dist,tangent,normal = load_flowline(cfg)
log.info("Loaded flowline")
h_obs,u_obs,ux_err,uy_err = load_fields(cfg,Q_c,V)
log.info("Loaded Remote Sensing Data")

bed = firedrake.Constant(-1.0e6)
s0 = icepack.compute_surface(thickness=h_obs,bed=bed,rho_I = rho_ice,rho_W = rho_sw)
b0 = firedrake.assemble(s0-h_obs)

if cfg.save_figs:
    Path(cfg.workspace,"out",cfg.shelf,cfg.name).mkdir(parents=True, exist_ok=True)
    if cfg.mesh.dim == 2:
        fig,ax = plot_mesh(mesh)
    elif cfg.mesh.dim == 3:
        fig,ax = plot_mesh(mesh2d)
    fig.savefig(Path(cfg.workspace,"out",cfg.shelf,cfg.name,"mesh.png"))

    fig,ax = tripcolor(h_obs,title = "Initial thickness")
    plot_flowlines(coordinates,ax=ax)
    fig.savefig(Path(cfg.workspace,"out",cfg.shelf,cfg.name,"initial_thickness.png"))
    fig,ax = tripcolor(u_obs,title = "Initial velocity")
    #ip_streamplot(u0,title = "Final velocity",density = 1000,percision=10,ax=ax)
    fig.savefig(Path(cfg.workspace,"out",cfg.shelf,cfg.name,"initial_velocity.png"))
    fig,axes = plot_all_1d(coordinates,dist,[h_obs,u_obs,[s0,b0]],labels=["Thickness","Velocity","Profile"])
    fig.savefig(Path(cfg.workspace,"out",cfg.shelf,cfg.name,"initial_1d.png"))
A0 = icepack.rate_factor(firedrake.Constant(263.15)) #fluidity


def viscosity(**kwargs):
    print("not here")
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    theta = kwargs["log_fluidity"]

    A = A0 * firedrake.exp(theta)
    return icepack.models.viscosity.viscosity_depth_averaged(
        velocity=u, thickness=h, fluidity=A
    )


#model = icepack.models.HybridModel(viscosity=viscosity)
#model = icepack.models.IceStream(viscosity=viscosity)
model = icepack.models.IceShelf(viscosity=viscosity)


opts = {
    "dirichlet_ids": [1,2,4],
    #"side_wall_ids":[],

    "diagnostic_solver_type": "petsc",
    "diagnostic_solver_parameters": {
        "snes_type": "newtontr",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}
solver = icepack.solvers.FlowSolver(model, **opts) 
print(model)
print(solver)
print(viscosity)

In [ ]:
u0 = solver.diagnostic_solve(
        velocity= u_obs, 
        thickness=h_obs, 
        surface = s0,
        log_fluidity = firedrake.Function(Q_c),
    )

In [ ]:
# model_kwargs = {}
# diagnostic_solver_kwargs = {}
# if cfg.icepack_model.laws is not None:
#     if "log_fluidity" in cfg.icepack_model.laws.keys():
#         # model_kwargs["viscosity"] = make_log_fluidity_viscosity_law(icepack_model.laws.log_fluidity.temp,Q_c)
#         # diagnostic_solver_kwargs["log_fluidity"] = firedrake.Function(Q_c)
#         A0 = icepack.rate_factor(firedrake.Constant(cfg.icepack_model.laws.log_fluidity.temp))

#         def viscosity(**kwargs):
#             print("here")
#             u = kwargs["velocity"]
#             h = kwargs["thickness"]
#             theta = kwargs["log_fluidity"]

#             A = A0 * firedrake.exp(theta)
#             return icepack.models.viscosity.viscosity_depth_averaged(
#                 velocity=u, thickness=h, fluidity=A
#             )
#         model_kwargs["viscosity"] = viscosity
#         diagnostic_solver_kwargs["log_fluidity"] = firedrake.Function(Q_c)


# if cfg.icepack_model.name == "shelf":
#     print(model_kwargs)
#     print(diagnostic_solver_kwargs)
#     model = icepack.models.IceShelf(viscosity = model_kwargs["viscosity"])
# elif cfg.icepack_model.name == "stream":
#     model = icepack.models.IceStream(**model_kwargs)
# elif cfg.icepack_model.name == "hybrid":
#     model = icepack.models.HybridModel(**model_kwargs)


model,diagnostic_solver_kwargs = icepack_model(cfg.icepack_model,Q_c,V)
logging.info(diagnostic_solver_kwargs)
log.info("Created model")
# solver = icepack_solver(cfg.flow_solver,model)
opts2 = icepack_solver(cfg.flow_solver,model)
print(opts)
print(opts2)
opts3 = {}
for key in opts2:
    opts3[key] = opts2[key]
solver = icepack.solvers.FlowSolver(model, **opts2) 
print(opts==opts2)
log.info("Created solver")
print(model.viscosity)
print(solver)
firedrake.Function(Q_c)

#print(diagnostic_solver_kwargs)

In [ ]:
u0 = solver.diagnostic_solve(
        velocity= u_obs, 
        thickness=h_obs, 
        surface = s0,
        log_fluidity = firedrake.Function(Q_c),
    )

In [ ]:
true_lf = firedrake.Function(Q_c)
print(true_lf)

In [ ]:
def add(a=1,b=2):
    return a+b

test_kwargs = {"a":0,"b":20}
print(add(**test_kwargs))

In [ ]:
dQxdx = firedrake.interpolate((h_obs*u0).dx(0)[0],Q_c)
dQydy = firedrake.interpolate((h_obs*u0).dx(1)[1],Q_c)



In [ ]:
flux_divergence = firedrake.interpolate(firedrake.as_vector((dQxdx,dQydy)),V)

In [ ]:
from preproc.utils.meshing import create_flowline_mesh,create_flowtube_mesh
import pandas as pd

df = pd.read_csv(Path("../data/Ekstrom/flowtube_test.csv"))
xleft = df['xleft'].dropna().to_numpy()
yleft = df['yleft'].dropna().to_numpy()
xright = df['xright'].dropna().to_numpy()
yright = df['yright'].dropna().to_numpy()

create_flowtube_mesh(xleft,yleft,xright,yright,Path("../data/Ekstrom"),Path("flowtube_test.geo"))



In [ ]:
coordinates = np.array([(x, y) for x,y in zip(df['xdata'].to_numpy(),df['ydata'].to_numpy())])

xdata,ydata = df['xdata'].dropna().to_numpy(),df['ydata'].dropna().to_numpy()
coordinates,dist = get_flowline(xdata,ydata,**cfg.flowline.smoothing)
x = coordinates[:,0]
y = coordinates[:,1]

create_flowline_mesh(x,y,Path("../data/Ekstrom"),Path("flowline_test.geo"))